In [1]:
import tensorflow as tf

/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/space/anaconda3/env

In [3]:
num_filters = 25
num_convs = 3
num_hidden_layers = 3
size_hidden_layers = 64

X = tf.placeholder(tf.float32, shape=(80, 5, 4, 20)) #(5, 4, 20) statespace size 80 is batch size

In [4]:
conv_out = tf.layers.conv2d(
    inputs=X,
    filters=num_filters,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.leaky_relu,
    name="conv_initial"
)

for i in range(0, num_convs - 1):
    padding = "same" if i < num_convs - 2 else "valid"
    conv_out = tf.layers.conv2d(
        inputs=conv_out,
        filters=num_filters,
        kernel_size=[3, 3],
        padding=padding,
        activation=tf.nn.leaky_relu,
        name="conv_{}".format(i)
    )

out = tf.layers.flatten(conv_out)
for _ in range(num_hidden_layers):
    out = tf.layers.dense(out, size_hidden_layers,
                          activation=tf.nn.leaky_relu)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


In [14]:
def foo(a, b):
    z = 10
    def bar():
        nonlocal z
        print(z)
        z += 10
        print(z)
        return 10
    return bar

In [15]:
foo(2,3)()

10
20


10

In [6]:
def num_tf_params():
    """Prints number of trainable parameters defined"""
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
            total_parameters += variable_parameters
    print(total_parameters)
num_tf_params()

43091


In [32]:
my_dict = {}
my_dict.update({1: 1})
my_dict.update({1: 2})

In [33]:
my_dict

{1: 2}

In [1]:
from human_aware_rl.baselines_utils import get_vectorized_gym_env
from overcooked_ai_py.mdp.overcooked_env import OvercookedEnv
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld
from human_aware_rl.baselines_utils import RewardShapingEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
import gym

/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/space/anaconda3/env

In [2]:
mdp_params = {
                'layout_name': 'simple',
                'start_order_list': None,
                'rew_shaping_params': {'PLACEMENT_IN_POT_REW': 3, 'DISH_PICKUP_REWARD': 3, 'SOUP_PICKUP_REWARD': 5, 'DISH_DISP_DISTANCE_REW': 0, 'POT_DISTANCE_REW': 0, 'SOUP_DISTANCE_REW': 0}
             }

env_params = {'horizon': 400}
params = {
    'mdp_params': mdp_params,
    'env_params': env_params,
#     'sim_threads': 2,
    'RUN_TYPE': "ppo"
}

mdp = OvercookedGridworld.from_layout_name(**params["mdp_params"])
env = OvercookedEnv(mdp, **params["env_params"])
featurize_fn=lambda x: mdp.lossless_state_encoding(x)

def gym_env_fn():
    gym_env = gym.make('Overcooked-v0')
    if params["RUN_TYPE"] == "joint_ppo":
        # If doing joint training, action space will be different (^2 compared to single agent training)
        gym_env.custom_init(env, joint_actions=True,
                            featurize_fn=featurize_fn, baselines=True)
    else:
        gym_env.custom_init(
            env, featurize_fn=featurize_fn, baselines=True)
    return gym_env
single_env = gym_env_fn()

# box (self.low.min(), self.high.max(), self.shape, self.dtype)
# shape (5, 4, 20)
print(single_env.observation_space) 

Box(0.0, 20.0, (5, 4, 20), float32)


/home/space/anaconda3/envs/cs6244-project/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [114]:
def add_tax_and_shipping(price):
    """Add tax and shipping
    """    
    
    tax = tf.get_variable("tax", (), dtype=tf.float32,
                          initializer=tf.constant_initializer(0.05))  # 0.05%
    shipping = tf.get_variable("shipping", (), dtype=tf.float32,
                            initializer=tf.constant_initializer(10.0))  # $10    

    tax = tf.Print(tax, [tax], message="Fuck you")
    total_price = price * (1.0 + tax) + shipping
        
    return total_price

In [113]:
import sys

def main():
    with tf.variable_scope("other_charge", reuse=tf.AUTO_REUSE) as scope:
        book_price = add_tax_and_shipping(10.0)
        sketchpad_price = add_tax_and_shipping(20.0)    
    init_op = tf.global_variables_initializer()  # Set up operator to assign all init values to variables    
    
    with tf.Session() as s:
        s.run(init_op)  # Actually assign initial value to variables        
        
        price = s.run(book_price)
        print("Price of the book: %f" % (price))        
    
        price = s.run(sketchpad_price)
        print("Price of the sketchpad: %f" % (price))    # Try to double tax!
    
    graph = tf.get_default_graph()
    tax = graph.get_tensor_by_name("other_charge/tax:0")    
    print("Doubling tax")
    double_tax = tf.assign(tax, tax * 2.0)
    
    with tf.Session() as s:
        s.run(init_op)  # Actually assign initial value to variables
        s.run(double_tax)
            
        price = s.run(book_price)
        print("Price of the book: %f" % (price))        
        price = s.run(sketchpad_price)
        print("Price of the sketchpad: %f" % (price))

In [101]:
sess = tf.Session()
with sess.as_default():
    tensor = tf.range(10)
    print_op = tf.print(tensor)
    with tf.control_dependencies([print_op]):
        out = tf.add(tensor, tensor)
    sess.run(out)

In [117]:
main()

Price of the book: 20.500000
Price of the sketchpad: 31.000000
Doubling tax
Price of the book: 21.000000
Price of the sketchpad: 32.000000


In [118]:
for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
    print(i)   # i.name if you want just a name

<tf.Variable 'conv_initial/kernel:0' shape=(5, 5, 20, 25) dtype=float32_ref>
<tf.Variable 'conv_initial/bias:0' shape=(25,) dtype=float32_ref>
<tf.Variable 'conv_0/kernel:0' shape=(3, 3, 25, 25) dtype=float32_ref>
<tf.Variable 'conv_0/bias:0' shape=(25,) dtype=float32_ref>
<tf.Variable 'conv_1/kernel:0' shape=(3, 3, 25, 25) dtype=float32_ref>
<tf.Variable 'conv_1/bias:0' shape=(25,) dtype=float32_ref>
<tf.Variable 'dense/kernel:0' shape=(150, 64) dtype=float32_ref>
<tf.Variable 'dense/bias:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'dense_1/kernel:0' shape=(64, 64) dtype=float32_ref>
<tf.Variable 'dense_1/bias:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'dense_2/kernel:0' shape=(64, 64) dtype=float32_ref>
<tf.Variable 'dense_2/bias:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'other_charge/tax:0' shape=() dtype=float32_ref>
<tf.Variable 'other_charge/shipping:0' shape=() dtype=float32_ref>


In [135]:
import pickle
# with open('/home/space/Github/cs6244-project/step_dict.pkl', 'rb') as handle:
#     targ_dict = pickle.load(handle)

with open('/home/space/Github/cs6244-project/ppo2_slices.pkl', 'rb') as handle:
    slices = pickle.load(handle)

In [146]:
# 0:obs, 1:returns, 2:masks, 3:actions, 4:values, 5:neglogpacs
slices[0].shape
slices[1].shape
slices[4].shape

(80,)

In [151]:
slices[4]

array([0.32372445, 0.27265075, 0.33076563, 0.30054137, 0.31540653,
       0.38281867, 0.34678856, 0.25838655, 0.27227175, 0.30964288,
       0.27949145, 0.33674482, 0.32170597, 0.34184968, 0.3661455 ,
       0.33271924, 0.3113713 , 0.37650043, 0.33786356, 0.38796183,
       0.3680344 , 0.37551874, 0.33624572, 0.27727935, 0.3470393 ,
       0.2574189 , 0.2795708 , 0.36556718, 0.37551874, 0.32566717,
       0.27279112, 0.293265  , 0.37818536, 0.32976893, 0.36590424,
       0.32545403, 0.31845722, 0.37117746, 0.30463475, 0.38244563,
       0.30681056, 0.33703414, 0.2794098 , 0.3379752 , 0.37882438,
       0.26318234, 0.37551874, 0.30454573, 0.33562243, 0.39674556,
       0.317602  , 0.29733938, 0.34832287, 0.40196285, 0.34038517,
       0.38265267, 0.3782602 , 0.348781  , 0.36490828, 0.35826206,
       0.3385953 , 0.3928745 , 0.34511355, 0.35877404, 0.39462367,
       0.39148033, 0.3401316 , 0.3603846 , 0.3658044 , 0.33541465,
       0.3043552 , 0.40549496, 0.27981564, 0.36293164, 0.29597

In [134]:
def f1(*args):
    for name in args:
        print("Hello", name)
f1((1,2,3))

Hello (1, 2, 3)


In [153]:
from human_aware_rl.ppo.ppo import get_ppo_agent

In [155]:
# /home/space/Github/cs6244-project/human_aware_rl/experiments/ppo_bc_experiments.py
agent_ppo_bc_train, ppo_config =  get_ppo_agent('ppo_bc_train_simple', 9456, best=True)

INFO:tensorflow:Restoring parameters from /home/space/Github/cs6244-project/checkpoints/ppo_runs/ppo_bc_train_simple/seed9456/best/variables/variables


In [157]:
agent_ppo_bc_train.__dict__

{'state_policy': <function human_aware_rl.baselines_utils.get_model_policy.<locals>.state_policy>,
 'direct_policy': <function human_aware_rl.baselines_utils.get_model_policy.<locals>.encoded_state_policy>,
 'history': [],
 'stochastic': True,
 'action_probs': False}

In [17]:
my_dict = {}
for k,v in my_dict.items():
    print(k,v)

In [8]:
tuple([1,2,3])

(1, 2, 3)

In [16]:
import numpy as np
np.array(True)

array(True)